---
export HOPSWORKS_HOST=60342400-fd68-11ee-a374-5db5bf1f1917.cloud.hopsworks.ai

export HOPSWORKS_PROJECT_NAME=Bytewax_pipeline

export HOPSWORKS_API_KEY=NJ9njtdzAdDmoAdL.VcPV9NacRISOXnsRVOglr8fQM9HhsiujewZJDzOzeBHlPyTjNV7Z73tYL7BxNGXJ

export FEATURE_GROUP_NAME=interactions

export FEATURE_GROUP_VERSION=1

---
python 1_kafka_topic.py

python 2_simulation.py

python -m bytewax.run "3_streaming_pipeline:setup_dataflow('$FEATURE_GROUP_NAME', $FEATURE_GROUP_VERSION, '$HOPSWORKS_HOST', '$HOPSWORKS_PROJECT_NAME', '$HOPSWORKS_API_KEY')" 

RUST_BACKTRACE=1 python -m bytewax.run "3_streaming_pipeline:setup_dataflow('$FEATURE_GROUP_NAME', $FEATURE_GROUP_VERSION, '$HOPSWORKS_HOST', '$HOPSWORKS_PROJECT_NAME', '$HOPSWORKS_API_KEY')"

---

In [22]:
import json
import sys
sys.path.insert(1, '../')

import pandas as pd
from tqdm import tqdm
import hopsworks
from confluent_kafka import Producer

import config
from utils.hsfs_bytewax import get_kafka_config
from features.users import generate_users
from features.videos import generate_video_content
from features.interactions import generate_interactions

def simulate_interactions():
    # Generate data for users
    user_data = generate_users(config.USERS_AMOUNT_PIPELINE)
    data_users_df = pd.DataFrame(user_data)

    # Generate data for videos
    video_data = generate_video_content(config.VIDEO_AMOUNT_PIPELINE)
    data_video_df = pd.DataFrame(video_data)

    # Generate interactions
    interactions = generate_interactions(
        config.INTERACTIONS_AMOUNT_PIPELINE, 
        user_data, 
        video_data,
    )
    data_interactions_df = pd.DataFrame(interactions)
    
    data_interactions_df['json'] = data_interactions_df.apply(lambda x: x.to_json(), axis=1)
    
    return [json.loads(i) for i in data_interactions_df.json.values]

data = simulate_interactions()
data[:3]

[{'interaction_id': '1422-91-0556',
  'user_id': 'OU595D',
  'video_id': '2ZU94X',
  'interaction_type': 'skip',
  'watch_time': 87,
  'interaction_date': '2024-04-18 14:20:59'},
 {'interaction_id': '1892-80-4966',
  'user_id': 'GC019W',
  'video_id': '4WO41I',
  'interaction_type': 'skip',
  'watch_time': 122,
  'interaction_date': '2024-04-18 14:20:59'},
 {'interaction_id': '3044-36-7740',
  'user_id': 'IG087J',
  'video_id': '1KS47H',
  'interaction_type': 'view',
  'watch_time': 20,
  'interaction_date': '2024-04-18 14:20:59'}]

In [16]:
# Connect to Hopsworks
project = hopsworks.login()
fs = project.get_feature_store()

kafka_api = project.get_kafka_api()
kafka_config = get_kafka_config(fs.id)

print(kafka_config)
producer = Producer(kafka_config)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://60342400-fd68-11ee-a374-5db5bf1f1917.cloud.hopsworks.ai/p/119
Connected. Call `.close()` to terminate connection gracefully.
{'bootstrap.servers': '172.16.4.25:9091', 'security.protocol': 'SSL', 'ssl.endpoint.identification.algorithm': 'none', 'ssl.ca.location': '/tmp/kafka_sc_119_-1_ca_chain.pem', 'ssl.certificate.location': '/tmp/kafka_sc_119_-1_client_cert.pem', 'ssl.key.location': '/tmp/kafka_sc_119_-1_client_key.pem'}


In [17]:
# Simulate interactions
interactions_data = simulate_interactions()
interactions_data[0]

{'interaction_id': '9812-78-8238',
 'user_id': 'RR951P',
 'video_id': '7YP51D',
 'interaction_type': 'skip',
 'watch_time': 157,
 'interaction_date': '2024-04-18 11:33:30'}

In [19]:
# Send to source topic
for interaction in tqdm(interactions_data[:10], desc="Sending messages"):
    print(interaction)
    producer.produce(
        config.KAFKA_TOPIC_NAME,
        json.dumps(interaction),
    )
    producer.flush()

Sending messages:  30%|███       | 3/10 [00:00<00:00, 21.78it/s]

{'interaction_id': '9812-78-8238', 'user_id': 'RR951P', 'video_id': '7YP51D', 'interaction_type': 'skip', 'watch_time': 157, 'interaction_date': '2024-04-18 11:33:30'}
{'interaction_id': '2438-26-4753', 'user_id': 'ZC342Y', 'video_id': '0HP06X', 'interaction_type': 'view', 'watch_time': 98, 'interaction_date': '2024-04-18 11:33:30'}
{'interaction_id': '6795-70-8245', 'user_id': 'AL852G', 'video_id': '0TD02F', 'interaction_type': 'skip', 'watch_time': 141, 'interaction_date': '2024-04-18 11:33:30'}
{'interaction_id': '1930-44-8804', 'user_id': 'IA528X', 'video_id': '8KA77T', 'interaction_type': 'like', 'watch_time': 22, 'interaction_date': '2024-04-18 11:33:30'}
{'interaction_id': '1584-76-4537', 'user_id': 'VZ873S', 'video_id': '6MY59E', 'interaction_type': 'skip', 'watch_time': 74, 'interaction_date': '2024-04-18 11:33:30'}


Sending messages:  90%|█████████ | 9/10 [00:00<00:00, 21.51it/s]

{'interaction_id': '8360-83-9259', 'user_id': 'LF197M', 'video_id': '8GS13E', 'interaction_type': 'skip', 'watch_time': 85, 'interaction_date': '2024-04-18 11:33:30'}
{'interaction_id': '8936-72-7250', 'user_id': 'CW504L', 'video_id': '0KS93V', 'interaction_type': 'skip', 'watch_time': 40, 'interaction_date': '2024-04-18 11:33:30'}
{'interaction_id': '7234-31-3381', 'user_id': 'OP675X', 'video_id': '9YA05J', 'interaction_type': 'skip', 'watch_time': 35, 'interaction_date': '2024-04-18 11:33:30'}
{'interaction_id': '4106-34-5958', 'user_id': 'RF538A', 'video_id': '9HW87T', 'interaction_type': 'skip', 'watch_time': 43, 'interaction_date': '2024-04-18 11:33:30'}
{'interaction_id': '1759-83-8126', 'user_id': 'SG073W', 'video_id': '5FH42R', 'interaction_type': 'like', 'watch_time': 64, 'interaction_date': '2024-04-18 11:33:30'}


Sending messages: 100%|██████████| 10/10 [00:00<00:00, 21.66it/s]


---